## Spam Filter with Naive Bayes

![Image](https://ampjar.com/wp-content/uploads/2019/07/HERO-avoid-spam-filters@2x.png)

### Introduction

To classify messages as spam or non-spam using the naive bayes algorithm, we basically need follow these steps:

* Learns how humans classify messages
* Uses that human knowledge to estimate probabilities for new messages (spam and non-spam)
* Classifies a new message based on these probability values

In this project we'll be working with the data set provided by [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can download data set and readme download from this link and also get data set information.

There are 5,572 SMS messages that are already classified by humans. Let's read it and take a quick look on the first five rows.

In [1]:
import pandas as pd

classified_sms = pd.read_csv('SMSSpamCollection',
                            sep='\t',
                            header=None,
                            names=['Label', 'SMS'])

classified_sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Let's find now what percentage of the messages is spam and what isn't ("ham" means non-spam).

In [2]:
total_persentages = classified_sms['Label'].value_counts(normalize=True)

print('''There are {:.1%} spam messages and {:.1%} non-spam messages in the data set'''
      .format(total_persentages[1], total_persentages[0]))

There are 13.4% spam messages and 86.6% non-spam messages in the data set


### Building the filter

#### Test preparation

When creating software, it's a good rule to think over tests in advance. In this case we should split our data set into two parts:
* training set (about 80% of the dataset)
* test set (about 20% of the dataset)

We'll create the spam filter using training set only. Once we have it, we'll apply the algorithm on testing set and compare  results with the human classificatioan. Our goal here is accuracy greater than **80%**.

To ensure that spam and ham messages are spread properly throughout the datasets we'll randomize all the data.

In [3]:
#Randomize whole data set
classified_sms.sample(frac=1, random_state=1)

#Index to split with
split_on = round(len(classified_sms) * 0.8)
train_set = classified_sms.iloc[:split_on, :].copy()
test_set = classified_sms.iloc[split_on:, :].copy()

#Reset index
train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)

Let's check the spam and ham ratio in these new sets. It should be similar to what we have in the full data set.

In [4]:
train_persentages = train_set['Label'].value_counts(normalize=True)
test_persentages = test_set['Label'].value_counts(normalize=True)

print('''Full data set: {:.1%} spam messages and {:.1%} non-spam messages
Train set: {:.1%} spam messages and {:.1%} non-spam messages
Test set: {:.1%} spam messages and {:.1%} non-spam messages'''
      .format(total_persentages[1], total_persentages[0],
             train_persentages[1], train_persentages[0],
             test_persentages[1], test_persentages[0]))

Full data set: 13.4% spam messages and 86.6% non-spam messages
Train set: 13.5% spam messages and 86.5% non-spam messages
Test set: 13.0% spam messages and 87.0% non-spam messages


Everything looks fine.

#### Data cleaning

We want to classify messages by the words in them so we need to convert them into sets of words. It means:
* Remove any punctuation
* Reduce everything to the lowercase
* Split the message

In [5]:
train_set['SMS'] = train_set['SMS'].str.replace(
    r'\W', ' ',regex=True).str.lower().str.split()

train_set

,Label,SMS
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."
...,...,...
4453,ham,"[i, ve, told, you, everything, will, stop, jus..."
4454,ham,"[or, i, guess, lt, gt, min]"
4455,ham,"[i, m, home, ard, wat, time, will, u, reach]"
4456,ham,"[storming, msg, wen, u, lift, d, phne, u, say,..."


Now we should create a list of unique words - the **vocabulary**.

In [6]:
vocabulary = []

for message in train_set['SMS']:
    for word in message:
        vocabulary.append(word)
        
#Remove duplicates and return the list
vocabulary_set = set(vocabulary)
vocabulary = list(vocabulary_set)

len(vocabulary)

7813

There are 7813 words in the vocabulary!

Now we can use it to count how many words are in each message. We'll use the dictionary for this purpose.

In [7]:
#Create dictionary with zeroes for every word from vocabulary
word_counts_per_sms = {unique_word: [0] * len(train_set['SMS']) for unique_word in vocabulary}

#Iterate through messages
for index, sms in enumerate(train_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

#Test
for word in vocabulary[:5]:
    print(word, word_counts_per_sms[word][:10])

kusruthi [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
shivratri [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
oru [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
aaooooright [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
laying [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


These results we'll combine with the trainig dataframe.

In [8]:
word_counts_df = pd.DataFrame(word_counts_per_sms)
train_set_clean = pd.concat([train_set, word_counts_df], axis=1)
train_set_clean.head()

,Label,SMS,kusruthi,shivratri,oru,aaooooright,laying,beth,hill,drinking,...,ntimate,musthu,associate,hotel,gives,brats,thk,workage,slowly,hor
0,ham,"[go, until, jurong, point, crazy, available, o...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[ok, lar, joking, wif, u, oni]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Probabilities

We've got cleaned training set and we are ready to go. Let's first recall the probabilities that we need to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham) \\
\end{equation}

We must calculate `p_spam` and `p_ham` and also probabilities for each word by the following formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Let's prepare some constant values first:

In [9]:
#Probabilities
p_spam = train_persentages[1]
p_ham = train_persentages[0]

#Number of words
n_spam = train_set_clean.loc[train_set_clean['Label'] == 'spam'].sum()[2:].sum()
n_ham = train_set_clean.loc[train_set_clean['Label'] == 'ham'].sum()[2:].sum()
n_voc = len(vocabulary)

# Laplace coef
alpha = 1

Now we move on probabilities themselves.

In [10]:
#Dictionaries for probabilities
words_given_spam = {unique_word: 0 for unique_word in vocabulary}
words_given_ham = {unique_word: 0 for unique_word in vocabulary}

for word in vocabulary:
    #How many times word appears in spam and ham messages
    n_word_spam = train_set_clean.loc[train_set_clean['Label'] == 'spam', word].sum()
    n_word_ham = train_set_clean.loc[train_set_clean['Label'] == 'ham', word].sum()
    
    #Probabilities
    p_word_given_spam = (n_word_spam + alpha
                        ) / (n_spam + alpha * n_voc)
    
    p_word_given_ham = (n_word_ham + alpha
                        ) / (n_ham + alpha * n_voc)
    
    #Update dictionaries
    words_given_spam[word] = p_word_given_spam
    words_given_ham[word] = p_word_given_ham
    
    
#Test
for word in vocabulary[:5]:
    print(word, words_given_spam[word], words_given_ham[word])

kusruthi 4.3189081800120926e-05 6.150250622712876e-05
shivratri 4.3189081800120926e-05 3.075125311356438e-05
oru 4.3189081800120926e-05 7.687813278391095e-05
aaooooright 4.3189081800120926e-05 3.075125311356438e-05
laying 4.3189081800120926e-05 3.075125311356438e-05


#### Filter itself

Once we've calculated all the parameters we need, we can create the spam filter. The filter is a function that:
* Takes in as input a new message **(w1, w2, ..., wn)**
* Calculates **P(Spam|w1, w2, ..., wn)** and **P(Ham|w1, w2, ..., wn)**
* Compares the values of **P(Spam|w1, w2, ..., wn)** and **P(Ham|w1, w2, ..., wn)**, and then:
    * If **P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn)**, then the message is classified as **ham**
    * If **P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn)**, then the message is classified as **spam**
    * If **P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn)**, then the algorithm may request **human** help
    
Let's write such function.

In [11]:
import re

def classify(message):
    #Clean message like train set
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    #Calculate probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in words_given_spam:
            p_spam_given_message *= words_given_spam[word]
            
        if word in words_given_ham:
            p_ham_given_message *= words_given_ham[word]
    
    #Print results
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
        
#Test function
classify('WINNER!! Send this secret code C3421 to unlock the money!')
classify("I'm on my way")

P(Spam|message): 7.608478331174781e-24
P(Ham|message): 3.335898472753021e-26
Label: Spam
P(Spam|message): 4.546575542387177e-17
P(Ham|message): 9.245128581483754e-12
Label: Ham


#### Test the algorithm

It worked with two examples, now it's time use the algorithm with the test set. Let's write similar function for this purpose.

Then we'll apply it to the test set and find the accuracy.

In [12]:
def classify_test_set(message):
    #Clean message like train set
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    #Calculate probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in words_given_spam:
            p_spam_given_message *= words_given_spam[word]
            
        if word in words_given_ham:
            p_ham_given_message *= words_given_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'need human classification'
    
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

correct = (test_set['predicted'] == test_set['Label']).sum()
accuracy = correct / len(test_set)
print('The filter accuracy - {:.2%}'.format(accuracy))

The filter accuracy - 98.56%


Wow, we've got superb result - **98.56%**! I've expected that accuracy to be worse than 80% to be honest.

### Conclusion

We've created a very efficient spam filter using the multinomial Naive Bayes algorithm. The result accuracy is **98.56%** which is great because we initially aimed for an accuracy of 80% only.

Further we could make more complex algorithm by including the letter case, for example.